In [3]:
import sys
from pyspark.sql.functions import expr
from pyspark.sql import Window
import pyspark.sql.functions as psf
from pyspark.sql.types import *
from pyspark.sql.functions import col
import pyspark.sql.functions as F
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer
import os
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.ml.classification import LogisticRegression

2024-04-21 00:16:09.421439: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 00:16:09.432455: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 00:16:09.503970: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-21 00:16:09.763592: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 00:16:11.660323: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [14]:
import tensorflow._api.v2.compat.v1 as tf
tf.disable_v2_behavior()

In [2]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (589.8 MB)


In [27]:
!pip install tensorflow==1.15

IOStream.flush timed out
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tensorflow==1.15 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==1.15


In [5]:
sc = pyspark.SparkContext.getOrCreate()

In [6]:
base_dir_train = "Train/"
base_dir_test = "Test/"

In [7]:
def load_images(folder_name, csv_file, base_dir, label):
    df = spark.read.csv(base_dir + csv_file, header=False, inferSchema=True).toDF("filename")
    df = df.withColumn("path", F.concat(lit(base_dir + folder_name + "/"), col("filename")))
    paths = df.select("path").rdd.flatMap(lambda x: x).collect()
    if paths:
        images_df = spark.read.format('image').load(paths)
        images_df = images_df.select("image.*")
    else:
        images_df = spark.createDataFrame([], schema=None)
    images_df = images_df.withColumn("label", F.lit(label))
    return images_df

In [8]:
covid_images_df_train = load_images("COVID19", "COVID19_split_Kausar.csv", base_dir_train, 1)
normal_images_df = load_images("NORMAL", "NORMAL_split_Kausar.csv", base_dir_train, 0)
pneumonia_images_df = load_images("PNEUMONIA", "PNEUMONIA_split_Kausar.csv", base_dir_train, 2)

2024-04-21 00:16:28,787 INFO (Thread-5-660439) Error while sending or receiving.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer
2024-04-21 00:16:28,789 INFO (Thread-5-660439) Closing down clientserver connection
2024-04-21 00:16:28,790 INFO (Thread-5-660439) Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 503, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt

In [9]:
data_frames = [normal_images_df, covid_images_df_train, pneumonia_images_df]

In [10]:
combined_df = reduce(DataFrame.unionAll, data_frames)

In [11]:
featurizer = DeepImageFeaturizer(inputCol="image", outputCol="features", modelName="ResNet50")

In [12]:
lr = LogisticRegression(maxIter=5, regParam=0.03, elasticNetParam=0.5, labelCol="label")

In [15]:
sparkdn = Pipeline(stages=[featurizer, lr])
spark_model = sparkdn.fit(combined_df)

AttributeError: module 'tensorflow' has no attribute 'Session'

In [ ]:
covid_images_df_train = load_images("COVID19", "COVID19_split_Kausar.csv", base_dir_train)
normal_images_df = load_images("NORMAL", "NORMAL_split_Kausar.csv", base_dir_train)
pneumonia_images_df = load_images("PNEUMONIA", "PNEUMONIA_split_Kausar.csv", base_dir_train)

In [10]:
train_files_pnemonia = spark.read.csv("Train/PNEUMONIA_split_Kausar.csv")
train_files_normal = spark.read.csv("Train/NORMAL_split_Kausar.csv")
train_files_covid = spark.read.csv("Train/COVID19_split_Kausar.csv")

In [11]:
test_files_pnemonia = spark.read.csv("Test/PNEUMONIA_split_Kausar.csv")
test_files_normal = spark.read.csv("Test/NORMAL_split_Kausar.csv")
test_files_covid = spark.read.csv("Test/COVID19_split_Kausar.csv")